In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In questa lezione, vedremo una panoramica sull'utilizzo di Keras e sulle due API offerte per la creazione di modelli per il deep learning.

## Parte 1: lettura dei dati

Al solito, iniziamo leggendo i dati che vogliamo utilizzare. In questo caso, useremo uno tra i dataset forniti di default con Keras (potete trovare un elenco completo[qui](https://keras.io/api/datasets/)), ovvero *MNIST*.

MNIST è un dataset che contiene un gran numero di immagini rappresentative delle cifre decimali; per i più attenti, lo abbiamo già utilizzato brevemente nelle operazioni di clustering con Scikit-Learn usando il metodo `load_digits`.

Specifichiamo anche due costanti, ovvero `NUM_CLASSES`, rappresentativa del numero di classi (ovvero 10), e `INPUT_SHAPE`, che indica le dimensioni del tensore in ingresso alla rete (ovvero, le dimensioni in pixel di ciascuna immagine).

> **Nota**: il valore di `INPUT_SHAPE` è pari a 28 (numero di pixel in altezza) per 28 (numero di pixel in larghezza) per 1 (numero di canali per un'immagine in bianco e nero). Se l'immagine fosse stata a colori, o RGB, avremmo avuto 3 canali.

In [2]:
NUM_CLASSES = 10
INPUT_SHAPE = (28, 28, 1)
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

### Parte 1.1: preprocessing

In questo caso, però, useremo la funzione [`load_data`](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data), e dovremo effettuare alcune semplici operazioni di preprocessing.

In particolare:

1. normalizzeremo i valori assunti dai pixel delle singole immagini in modo che ricadano tra 0 ed 1;
2. useremo la funzione [`expand_dims`](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html) di NumPy per fare in modo che sia aggiunta una nuova dimensione ai dati, in modo che rispettino il parametro `INPUT_SHAPE`;
3. infine, convertiremo le label in dati *categorical*, mediante la funzione `to_categorical` contenuta nel package `keras.utils`. In particolare, questa funzione effettua il *one-hot encoding* del vettore passato in ingresso.

In [3]:
# 1. Normalizzazione dei dati
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

# 2. Aggiunta del numero di canali
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
print(X_train.shape)

# 3. One hot encoding delle label
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)
print(y_train[:5])

(60000, 28, 28, 1)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## Parte 2: la Sequential API

Keras offre due diverse possibilità per creare un modello di rete neurale. 

La prima che tratteremo è la [**Sequential API**](https://keras.io/guides/sequential_model/), che modella la rete come una sequenza *strettamente lineare* di layer, ognuno dei quali ha esattamente un tensore in ingresso ed un tensore in uscita.

### Parte 2.1: creazione del modello

L'idea è quindi quella di creare una rete, di dimensioni *volutamente* ridotte, con una struttura di questo tipo:

1. un layer di [`Input`](https://keras.io/api/layers/core_layers/input), che si occupa dell'ingresso dei dati all'interno della rete;
2. due sequenze (alternate) fatte da un layer di convoluzione (layer [`Conv2d`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)) ed uno di max pooling (layer [`MaxPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)), utili ad estrarre due diversi "strati" di feature dall'immagine in ingresso;
3. un layer [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), che serve a "vettorizzare" le feature estratte prima della funzione di classificazione;
4. il layer di classificazione vero e proprio, dato da un layer completamente connesso ([`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) con funzione di attivazione `softmax`.

In [4]:
smodel = keras.Sequential([
    layers.Input(shape=INPUT_SHAPE),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(NUM_CLASSES, activation='softmax')
], name='sequential_model')

Possiamo vedere l'architettura della nostra rete usando la funzione `summary()`.

In [5]:
smodel.summary()

Model: "sequential_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                8010      
Total params: 17,578
Trainable params: 17,578
Non-trainable params: 0
______________________________________________

### Parte 2.2: addestramento del modello

Possiamo adesso passare a finalizzare la struttura del modello, specificando le funzioni di costo e di ottimizzazione mediante il metodo `compile`.

In particolare, useremo come funzione di costo la `categorical_crossentropy`, dato che si tratta di un problema multi-classe, mentre come ottimizzatore useremo la funzione `sgd`. Specifichiamo anche la metrica che andrà calcolata ad ogni iterazione (o *epoca*) di addestramento, ovvero l'accuracy sui dati di test.

In [6]:
smodel.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)

Addestriamo ora il modello usando il metodo `fit`.

In [7]:
smodel.fit(X_train, y_train, batch_size=64, epochs=10)

Epoch 1/10
938/938 [==============================] - 16s 17ms/step - loss: 1.5405 - accuracy: 0.5423
Epoch 2/10
938/938 [==============================] - 15s 16ms/step - loss: 0.2820 - accuracy: 0.9158
Epoch 3/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1930 - accuracy: 0.9444
Epoch 4/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1542 - accuracy: 0.9552
Epoch 5/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1308 - accuracy: 0.9616
Epoch 6/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1138 - accuracy: 0.9666
Epoch 7/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1011 - accuracy: 0.9704
Epoch 8/10
938/938 [==============================] - 15s 16ms/step - loss: 0.0916 - accuracy: 0.9724
Epoch 9/10
938/938 [==============================] - 15s 16ms/step - loss: 0.0909 - accuracy: 0.9724
Epoch 10/10
938/938 [==============================] - 16s 17ms/step - loss: 0.082

Come possiamo vedere, l'accuracy raggiunta dal modello dopo dieci epoche di addestramento è del 97.46%.

## Parte 3: la Functional API

La seconda possibilità offerta da Keras è quella relativa all'uso della [**Functional API**](https://keras.io/guides/functional_api), che ci offre maggior controllo sul modello da creare, al prezzo di una sintassi leggermente più complessa.

Rispetto alla Sequential API, la Functional API basa il suo funzionamento sul concetto di *grafo aciclico diretto*, il che permette quindi topologie più complesse, con layer condivisi, input ed output multipli, e così via. Per fare un esempio, sarebbe impossibile implementare un'architettura come [Inception](https://i.stack.imgur.com/iNy2U.png) senza usare un'API di questo tipo.

Vediamo quindi come creare un modello analogo al precedente usando questa funzionalità.

### Parte 3.1: creazione del modello

Notiamo che i layer che utilizzeremo *non cambiano*; cambia solo la sintassi e le funzioni utilizzate per concatenarli. Partiamo quindi dal layer di `Input`.

In [8]:
inputs = layers.Input(shape=INPUT_SHAPE)

A questo punto, aggiungiamo un nodo al *grafo* di layer che compone la rete neurale andando a "chiamare" un nuovo layer su quello precedente, e creando così un collegamento tra i due.

Dal punto di vista pratico, usiamo una sintassi del tipo:

```py
# architettura = nuovo_layer(layer_esistente)
```

che equivale a:

In [9]:
x = layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)

Ripetiamo questa operazione fino a raggiungere il layer `outputs`, che sarà quello della classificazione mediante `softmax`.

In [10]:
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.Conv2D(32, kernel_size=(3,3), activation='relu')(x)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

Creiamo adesso un nuovo oggetto di tipo [`Model`](https://www.tensorflow.org/api_docs/python/tf/keras/Model). Ricordiamo di definire i layer di input ed output per il modello e, opzionalmente, specificare un nome.

Verifichiamo inoltre con `summary()` che la struttura sia coerente a quella vista in precedenza.

In [11]:
fmodel = keras.Model(inputs=inputs, outputs=outputs, name='functional_model')
fmodel.summary()

Model: "functional_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)           

Una nota: vediamo che, a differenza del modello ottenuto mediante Sequential API, qui siamo in grado di vedere la forma del layer di input. Questo effetto è *voluto*.

In ultimo, compiliamo il modello ed addestriamolo alla stessa maniera del precedente.

In [12]:
fmodel.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)

fmodel.fit(X_train, y_train, batch_size=64, epochs=10)

Epoch 1/10
938/938 [==============================] - 16s 16ms/step - loss: 1.4128 - accuracy: 0.5809
Epoch 2/10
938/938 [==============================] - 15s 17ms/step - loss: 0.2863 - accuracy: 0.9143
Epoch 3/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1955 - accuracy: 0.9425
Epoch 4/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1527 - accuracy: 0.9541
Epoch 5/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1274 - accuracy: 0.9623
Epoch 6/10
938/938 [==============================] - 16s 17ms/step - loss: 0.1144 - accuracy: 0.9664
Epoch 7/10
938/938 [==============================] - 16s 17ms/step - loss: 0.1041 - accuracy: 0.9683
Epoch 8/10
938/938 [==============================] - 15s 16ms/step - loss: 0.0955 - accuracy: 0.9720
Epoch 9/10
938/938 [==============================] - 16s 17ms/step - loss: 0.0897 - accuracy: 0.9727
Epoch 10/10
938/938 [==============================] - 16s 17ms/step - loss: 0.086

Ovviamente, le prestazioni sono molto simili, trattandosi, nei fatti, dello stesso modello.